In [4]:
!pip install PyMuPDF transformers langchain torchvision

In [6]:
from transformers import BartTokenizer, BartForConditionalGeneration
import fitz
import torch

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')


===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {WindowsPath('C')}
The following directories listed in your path were found to be non-existent: {WindowsPath('/matplotlib_inline.backend_inline'), WindowsPath('module')}
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
The following directories listed in your path were found to be non-existent: {WindowsPath('/usr/local/cuda/lib64')}
DEBUG: Possible options found for libcudart.so: set()
CUDA SETUP: PyTorch settings found: CUDA_VERSION=118, Highest Compute Capability: 7.5.
CUDA SETUP: To manually override the PyTorch CUDA version please see:https://github.com/TimDettmers/bitsandbytes/blob/main/how_to_use_nonpytorch_cuda.md
CUDA SETUP: Loading binary C:\Users\Siddhanth\anaconda3\envs\gpu_environment\lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.so...
argument of type 'Windo

RuntimeError: Failed to import transformers.models.bart.modeling_bart because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):

        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues

In [ ]:
import fitz
def extract_text(pdf_path):
  pdf_document = fitz.open(pdf_path)
  text = ""
  for page_number in range(len(pdf_document)):
    page = pdf_document.load_page(page_number)
    text+=page.get_text()
  pdf_document.close()
  return text

resume_text = extract_text("Idhuvum resume.pdf")

In [ ]:
resume_text

In [ ]:
def summarize_text(text):
  inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
  summary_ids = model.generate(
      inputs.input_ids,
      num_beams=4,
      min_length=100,
      max_length=250,
      length_penalty=2.0
  )
  summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  return summary

summary = summarize_text(resume_text)
print(summary)

In [ ]:
!pip install -q accelerate bitsandbytes einops

In [ ]:
!nvidia-smi

In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM

import torch

model_name = "tiiuae/falcon-7b-instruct"

tokenizer=AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", revision="main", cache_dir="models/")

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

In [ ]:
llm = HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})

In [ ]:
from langchain import PromptTemplate, LLMChain
template = """
You are an intellegent Model. Generate questions based on the given input. I want you to generate questions from the summary that i've provided you. along with the answers.
Question: {summary}
Answer: """

prompt = PromptTemplate(template=template, input_variables=['summary'])

llm_chain = LLMChain(prompt=prompt, llm=llm)

#question = " I want you to generate questions from the summary that i've provided you. along with the answers"

print(llm_chain.run(summary))